In [1]:
import torch
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch import nn
import torch.nn.functional as F

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


Prepare Train / test data sets

In [2]:
num_workers = 0
batch_size = 32
valid_size = 0.2

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform)
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform)

num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified


## Simple CNN with two convolutions

7 (weighted) layer 이상을 쌓으려면... pool 한번에 절반씩 줄어들기 때문에 pool을 계속 하는 것도 이상함. pool 네 번이면 2x2로 작아짐. 

pooling 없이 conv만 하면? -- pooling이나 conv나 계산량이 비슷하므로 conv를 자꾸 하는 것도 나쁘지 않을 듯. 

In [3]:
import copy


#activation = [F.relu, approx_relu][1]

def approx_relu(x):
    """
    Refer to Table 1 of
    https://openreview.net/attachment?id=rkxsgkHKvH&name=original_pdf
    """
    return 0.47 + 0.5*x + 0.09*x**2


activation = approx_relu
    
class Net(nn.Module):    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 8, 3)
        self.conv2 = nn.Conv2d(8, 8, 3)
        self.bn1 = nn.BatchNorm2d(8)
        self.pool = nn.AvgPool2d(2, 2)
        self.conv3 = nn.Conv2d(8, 16, 3)
        self.conv4 = nn.Conv2d(16, 16, 3)
        self.bn2 = nn.BatchNorm2d(16)
        
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = activation(self.bn1(self.conv2(x)))
        x = self.pool(x)
        x = self.conv3(x)
        x = activation(self.bn2(self.conv4(x)))
        x = self.pool(x)
        x = x.view(-1, 16 * 5 * 5)
        x = activation(self.fc1(x))
        x = activation(self.fc2(x))
        x = self.fc3(x)
        return x

# create a complete CNN
model = Net()
model.to('cuda')
train_on_gpu=True

print(model)

Net(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1))
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


## Optimizer

In [4]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=.001, momentum=0.95)#, weight_decay=0.09)# WD makes 
#ptimizer = optim.Adam(model.parameters(), lr=.004, weight_decay=0.9)

import matplotlib.pyplot as plt 

FastAI training

In [5]:
from fastai.vision.all import *
import torchvision

import fastai
from fastai.optimizer import OptimWrapper


from fastai.data.core import DataLoaders
from fastai.learner import Learner
from fastai.callback.progress import ProgressCallback
#from fastai.callback.data import CudaCallback


dls = DataLoaders(train_loader, test_loader)

learn = Learner(dls, model, loss_func=criterion)#, opt_func=opt_func)#, cbs=[CudaCallback])

learn.fine_tune(20)

epoch,train_loss,valid_loss,time
0,1.599134,1.581530,00:17


epoch,train_loss,valid_loss,time
0,1.444366,1.420980,00:17
1,1.373034,1.370201,00:17
2,1.336980,1.316324,00:17
3,1.258206,1.269635,00:17
4,1.218838,1.226007,00:17
5,1.140185,1.188145,00:17
6,1.023007,1.252276,00:17
7,1.016322,1.318512,00:17
8,0.925849,1.179548,00:17
9,0.897124,1.158281,00:17


## Training

In [40]:
n_epochs = 2
train_losslist=[]
valid_loss_min = np.Inf 

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    model.train()
    for data, target in train_loader:
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
        
    model.eval()
    for data, target in valid_loader:
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        output = model(data)
        loss = criterion(output, target)
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    train_losslist.append(train_loss)
        
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        #torch.save(model.state_dict(), 'model_cifar_2fc.pt')
        valid_loss_min = valid_loss
        

Epoch: 1 	Training Loss: 1.012313 	Validation Loss: 0.302897
Validation loss decreased (inf --> 0.302897).  Saving model ...
Epoch: 2 	Training Loss: 0.991354 	Validation Loss: 0.284357
Validation loss decreased (0.302897 --> 0.284357).  Saving model ...
Epoch: 3 	Training Loss: 0.974319 	Validation Loss: 0.278672
Validation loss decreased (0.284357 --> 0.278672).  Saving model ...
Epoch: 4 	Training Loss: 0.963115 	Validation Loss: 0.265368
Validation loss decreased (0.278672 --> 0.265368).  Saving model ...
Epoch: 5 	Training Loss: 0.951391 	Validation Loss: 0.267628
Epoch: 6 	Training Loss: 0.941690 	Validation Loss: 0.273839
Epoch: 7 	Training Loss: 0.935993 	Validation Loss: 0.250015
Validation loss decreased (0.265368 --> 0.250015).  Saving model ...
Epoch: 8 	Training Loss: 0.927553 	Validation Loss: 0.270340
Epoch: 9 	Training Loss: 0.918959 	Validation Loss: 0.253501
Epoch: 10 	Training Loss: 0.910461 	Validation Loss: 0.249090
Validation loss decreased (0.250015 --> 0.249090)

In [ ]:
torch.save(model.state_dict(), "Net_3fc2act.pt")

## Validation accuracy

In [116]:
# track test loss
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, 
    num_workers=num_workers)

test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()
# iterate over test data
for data, target in test_loader:
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    output = model(data)
    loss = criterion(output, target)
    test_loss += loss.item()*data.size(0)
    _, pred = torch.max(output, 1)    
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())

    for i in range(len(data)):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 2.059538

Test Accuracy of airplane: 71% (719/1000)
Test Accuracy of automobile: 79% (794/1000)
Test Accuracy of  bird: 53% (533/1000)
Test Accuracy of   cat: 48% (487/1000)
Test Accuracy of  deer: 64% (649/1000)
Test Accuracy of   dog: 54% (541/1000)
Test Accuracy of  frog: 77% (775/1000)
Test Accuracy of horse: 70% (704/1000)
Test Accuracy of  ship: 78% (782/1000)
Test Accuracy of truck: 74% (747/1000)

Test Accuracy (Overall): 67% (6731/10000)


## 성능 

(Conv - BN - Avg Pool)
conv 5x5, same padding = 67
conv 3x3, same padding = 68
conv 3x3, valid padding = 67



--------------------------

relu + maxpool: ~62%

relu + avgpool: ~58% -- OK, maxpool -> avgpool은 큰 문제 없음. 

approx. relu + avgpool: 52% !! 

approx. relu + avgpool + BN (2Conv + 3FC, 20 epoch): 58% 정도? 

approx. relu + avgpool + BN (2Conv + 2FC, 50 epoch: 59% 


## Number of trainable parameters

In [9]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(f"We have to train {params} parameters")

We have to train 62006 parameters
